In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2748f5a82b8bde1267666c1d8fbdf8a790850c0c2f868b49d1535d02b6259fea
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D, Conv1D
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
import yfinance as yf
import random
import os
from google.colab import drive
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
# import ta
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_s_and_p_tickers():
  s_and_p = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
  tickers = [symbol for symbol in s_and_p.Symbol.to_list() if str.isalpha(symbol)]
  return tickers

In [ ]:
tickers = get_s_and_p_tickers()

In [ ]:
# def calculate_bollinger_bands(data, window=10, num_of_std=2):
#     """Calculate Bollinger Bands"""
#     rolling_mean = data.rolling(window=window).mean()
#     rolling_std = data.rolling(window=window).std()
#     upper_band = rolling_mean + (rolling_std * num_of_std)
#     lower_band = rolling_mean - (rolling_std * num_of_std)
#     return upper_band, lower_band

# def calculate_rsi(data, window=10):
#     """Calculate Relative Strength Index"""
#     delta = data.diff()
#     gain = delta.clip(lower=0)
#     loss = -delta.clip(upper=0)
#     avg_gain = gain.rolling(window=window, min_periods=1).mean()
#     avg_loss = loss.rolling(window=window, min_periods=1).mean()
#     rs = avg_gain / avg_loss
#     rsi = 100 - (100 / (1 + rs))
#     return rsi

# def calculate_roc(data, periods=10):
#     """Calculate Rate of Change."""
#     roc = ((data - data.shift(periods)) / data.shift(periods)) * 100
#     return roc

In [ ]:
def get_all_available_indicators(sample_data):
    """Print all available technical indicators from ta package"""

    # Add all technical indicators
    data_with_all = ta.add_all_ta_features(
        sample_data,
        open="Open",
        high="High",
        low="Low",
        close="Close",
        volume="Volume",
        fillna=True
    )

    # Get new columns (indicators) added
    original_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    indicator_cols = [col for col in data_with_all.columns if col not in original_cols]

    # Print by category
    categories = {
        'volume': 'Volume',
        'volatility': 'Volatility',
        'trend': 'Trend',
        'momentum': 'Momentum',
        'others': 'Others'
    }

    print("Available Technical Indicators:")
    for prefix, category in categories.items():
        print(f"\n{category} Indicators:")
        indicators = [col for col in indicator_cols if col.startswith(prefix)]
        for ind in indicators:
            print(f"- {ind}")

    print(f"\nTotal number of indicators: {len(indicator_cols)}")

    return data_with_all

In [ ]:
def get_s_and_p(tickers):
  ticker_data_frames = {}
  #stats = {}

  remove = []
  for ticker in tickers:

      data = yf.download(ticker, period="5y", interval="1d")
      #financial_data = yf.Ticker(ticker)

      if data.empty:#or len(financial_data.financials.columns) < 2:
        remove.append(ticker)
        import time
        time.sleep(15)
        continue

      # financials = financial_data.financials[financial_data.financials.columns[1]]

      # if 'Total Revenue' not in financials or 'Gross Profit' not in financials or 'Net Income' not in financials:
      #   remove.append(ticker)
      #   continue


      # Calculate the daily percentage change
      close = data['Close'].squeeze() #just the daily close
      #upper, lower = calculate_bollinger_bands(close, window=14, num_of_std=2)
      #width = upper - lower
      #rsi = calculate_rsi(close, window=14)
      #roc = calculate_roc(close, periods=14)
      volume = data['Volume'].squeeze()
      high = data['High'].squeeze()
      low = data['Low'].squeeze()
      diff = data['Close'].diff(1).squeeze()
      percent_change_close = (data['Close'].pct_change() * 100).squeeze()
      volatility = (data['Close'].pct_change().std() * (252**0.5)).squeeze()
      rolling_average_close = data['Close'].rolling(window=20).mean().squeeze()
      time_index = np.arange(close.shape[0], 0, -1)

      #print(volatility)

      # revenue = financials['Total Revenue']
      # profit = financials['Gross Profit']
      # income = financials['Net Income']

      #width = width.squeeze()
      # rsi = rsi.squeeze()
      # roc = roc.squeeze()
      #print(time_index)

      #TODO expirement with different metrics
      ticker_df = pd.DataFrame({
          'close': close,
          'high': high,
          'low': low,
          'time_index': time_index,
          'volatility': volatility,
          'rolling_average_close': rolling_average_close,
          'percent_change_close': percent_change_close,
          # 'revenue': [revenue] * len(close),
          # 'profit': [profit] * len(close),
          # 'income': [income] * len(close),
          # 'gross_profit': np.full(shape=close.shape, fill_value=profit),
          # 'net_income': np.full(shape=close.shape, fill_value=income),
          #ticker+'_width': width,
          # ticker+'_rsi': rsi,
          # ticker+'_roc': roc,
          'volume': volume,
          'diff': diff,
          # 'percent_change_close': percent_change_close,
      })

      # MEAN = ticker_df.mean()
      # STD = ticker_df.std()

      # Keep track of mean and std
      # for column in MEAN.index:
      #   stats[f"{column}_mean"] = MEAN[column]
      #   stats[f"{column}_std"] = STD[column]

      # Normalize the training features
      #ticker_df = (ticker_df - MEAN) / STD #TODO removed normalization because using layernorm
      ticker_df.replace([np.inf, -np.inf], np.nan, inplace=True) #TODO check if needed
      ticker_df.dropna(inplace=True)

      ticker_data_frames[ticker] = ticker_df

  for ticker in remove:
    tickers.remove(ticker)

  return ticker_data_frames

In [ ]:
ticker_data_frames = get_s_and_p(tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
# For a specific ticker (e.g., 'MMM')
ticker = 'KEY'
num_columns = len(ticker_data_frames[ticker].columns)
print(f"Number of columns for {ticker}: {num_columns}")

Number of columns for KEY: 107


In [ ]:
df = ticker_data_frames[ticker]
for i in df.columns:
  if i == 'trailing_pe':
    print(df[i])

Date
2019-12-02 00:00:00+00:00   NaN
2019-12-03 00:00:00+00:00   NaN
2019-12-04 00:00:00+00:00   NaN
2019-12-05 00:00:00+00:00   NaN
2019-12-06 00:00:00+00:00   NaN
                             ..
2024-11-22 00:00:00+00:00   NaN
2024-11-25 00:00:00+00:00   NaN
2024-11-26 00:00:00+00:00   NaN
2024-11-27 00:00:00+00:00   NaN
2024-11-29 00:00:00+00:00   NaN
Name: trailing_pe, Length: 1258, dtype: float64


In [ ]:
from typing import Sequence
def zeroMaxScaler(data, train_percent):
  epsilon = 0.001
  max = np.maximum(np.max(data[0: math.floor(data.shape[0] * train_percent)], axis=(0, 1)).reshape(1, 1, -1), epsilon)
  return data / max


def create_sequences(
    dataframe_dict: dict,
    tickers: list,
    train_percent: float = .8,
    sequence_length: int = 20,
  ):

    epsilon = 1e-12

    #scaler = MinMaxScaler()
    scaler = StandardScaler()
    #scaler = MaxAbsScaler()

    sequence_dict = {}
    sequence_label_dict = {}
    for ticker in tickers:
      dataframe = dataframe_dict[ticker]
      #print(dataframe)
      data = dataframe.to_numpy()
      #print(data)
      N, D = data.shape
      L = sequence_length

      sequences = sliding_window_view(data, window_shape=L, axis=0)[:-1]

      mean = sequences.mean(axis=2, keepdims=True)
      std = sequences.std(axis=2, keepdims=True)

      normalized_sequences = (sequences - mean) / (std + epsilon)
      normalized_sequences = np.nan_to_num(normalized_sequences, nan=0)
      sequence_dict[ticker] = normalized_sequences

      close_index = dataframe.columns.get_loc('close')

      last_day_indices = np.arange(L - 1, N - 1)
      next_day_indices = last_day_indices + 1
      last_day_close_values = data[last_day_indices, close_index]

      next_day_close_values = data[next_day_indices, close_index]

      mean_close = mean[:, close_index, 0]
      std_close = std[:, close_index, 0]

      last_day_close = (last_day_close_values - mean_close) / (std_close + epsilon)
      next_day_close = (next_day_close_values - mean_close) / (std_close + epsilon)

      sequence_labels = np.column_stack((last_day_close, next_day_close, mean_close, std_close))
      sequence_label_dict[ticker] = sequence_labels

    return sequence_dict, sequence_label_dict


def train_test_split(
    ticker_data_frames: dict,
    tickers: list,
    train_percent: float = .8, #TODO increase
    sequence_length: int = 20,
):

  train_data = []
  train_labels = []
  dev_data = []
  dev_labels = []
  test_data = []
  test_labels = []

  #TODO double check all of this
  sequence_dict, label_dict = create_sequences(ticker_data_frames, tickers, train_percent, sequence_length)

  for ticker in sequence_dict.keys():
    sequence_len = len(sequence_dict[ticker])
    train_data.append(sequence_dict[ticker][:math.floor(sequence_len * train_percent)])
    hold_out_data = sequence_dict[ticker][math.floor(sequence_len * train_percent):]
    dev_data.append(hold_out_data[:len(hold_out_data) // 2])
    test_data.append(hold_out_data[len(hold_out_data) // 2:])

    train_labels.append(label_dict[ticker][:math.floor(sequence_len * train_percent)])
    hold_out_labels = label_dict[ticker][math.floor(sequence_len * train_percent):]
    dev_labels.append(hold_out_labels[:len(hold_out_data) // 2])
    test_labels.append(hold_out_labels[len(hold_out_labels) // 2:])

  train_data = np.concatenate(train_data, axis=0)
  train_labels = np.concatenate(train_labels, axis=0)
  dev_data = np.concatenate(dev_data, axis=0)
  dev_labels = np.concatenate(dev_labels, axis=0)
  test_data = np.concatenate(test_data, axis=0)
  test_labels = np.concatenate(test_labels, axis=0)

  return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [ ]:
train_data, train_labels, dev_data, dev_labels, test_data, test_labels = train_test_split(ticker_data_frames, tickers)

In [ ]:
def mae_loss(y_true, y_pred):
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)
    abs_error = tf.abs(y_true_next - y_pred_next)

    return tf.reduce_mean(abs_error)

def mse_loss(y_true, y_pred):
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)
    mse_error = (y_true_next - y_pred_next) ** 2

    return tf.reduce_mean(mse_error)

def dir_acc(y_true, y_pred):
    y_true_prev = tf.cast(y_true[:, 0], tf.float64)
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)

    true_change = y_true_next - y_true_prev
    pred_change = y_pred_next - y_true_prev

    correct_direction = tf.equal(tf.sign(true_change), tf.sign(pred_change))

    return tf.reduce_mean(tf.cast(correct_direction, tf.float64))


def percent_error_true(y_true, y_pred):
    epsilon = 1e-6
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)
    percent_error = abs(y_true_next - y_pred_next) / (y_true_next + epsilon)
    return tf.reduce_mean(tf.cast(percent_error, tf.float64))

def percent_error_adjusted(y_true, y_pred): #Use this one for 0 - 1 scaler
    threshold = 1e-3
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)
    percent_error = tf.where(
        y_true_next > threshold,
        abs(y_true_next - y_pred_next) / (y_true_next),
        tf.zeros_like(y_true_next)
    )
    return tf.reduce_mean(tf.cast(percent_error, tf.float64))

def percent_error_squared_adjusted(y_true, y_pred):
    threshold = 1e-3
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)
    percent_error = tf.where(
        y_true_next > threshold,
        abs(y_true_next - y_pred_next) / (y_true_next),
        tf.zeros_like(y_true_next)
    )
    return tf.reduce_mean(tf.cast(percent_error, tf.float64))

def percent_error_unnormalized(y_true, y_pred): #Use this one for std scaler
    threshold = 1e-3
    y_true_next = tf.cast(y_true[:, 1], tf.float64) * tf.cast(y_true[:, 3], tf.float64) + tf.cast(y_true[:, 2], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64) * tf.cast(y_true[:, 3], tf.float64) + tf.cast(y_true[:, 2], tf.float64)
    percent_error = tf.where(
        tf.math.abs(y_true_next) > threshold,
        abs(y_true_next - y_pred_next) / y_true_next,
        tf.zeros_like(y_true_next),
    )
    return tf.reduce_mean(percent_error)

def percent_error_squared_unnormalized(y_true, y_pred):
    threshold = 1e-3
    y_true_next = tf.cast(y_true[:, 1], tf.float64) * tf.cast(y_true[:, 3], tf.float64) + tf.cast(y_true[:, 2], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64) * tf.cast(y_true[:, 3], tf.float64) + tf.cast(y_true[:, 2], tf.float64)
    percent_error = tf.where(
        tf.math.abs(y_true_next) > threshold,
        (y_true_next - y_pred_next) / y_true_next,
        tf.zeros_like(y_true_next),
    )
    return tf.reduce_mean(tf.square(percent_error))


def error_direction(y_true, y_pred):
    y_true_next = tf.cast(y_true[:, 1], tf.float64)
    y_pred_next = tf.cast(y_pred[:, 0], tf.float64)
    error_direction = tf.sign(y_true_next - y_pred_next)
    return tf.reduce_mean(tf.cast(error_direction, tf.float64))

meanSquaredLogarithmicError = MeanSquaredLogarithmicError()

#TODO add absolute accuracy, or metric based on how much you would have made on a trade?

In [ ]:
# LSTM Baseline Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization

def build_lstm_model(input_shape,
                    lstm_units=50,
                    dropout_rate=0.2,
                    dense_units=25,
                    activation='relu',
                    output_units=1,
                    bidirectional=False,
                    num_lstm_layers=2):
    inputs = Input(shape=input_shape)
    x = inputs

    for _ in range(num_lstm_layers):
        if bidirectional:
            x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(x)
        else:
            x = LSTM(units=lstm_units, return_sequences=True)(x)
        x = Dropout(rate=dropout_rate)(x)

    # Optionally, add another LSTM layer without return_sequences
    if bidirectional:
        x = Bidirectional(LSTM(units=lstm_units, return_sequences=False))(x)
    else:
        x = LSTM(units=lstm_units, return_sequences=False)(x)
    x = Dropout(rate=dropout_rate)(x)

    x = BatchNormalization()(x)
    x = Dense(units=dense_units, activation=activation)(x)
    outputs = Dense(units=output_units, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
# # Model parameters
# input_shape = train_data.shape[1:]
# lstm_units = 50
# dropout_rate = 0.2
# dense_units = 25
# activation = 'relu'
# bidirectional = True
# num_lstm_layers = 5

# # Build the model
# model = build_lstm_model(input_shape, lstm_units=lstm_units,
#                          dropout_rate=dropout_rate, dense_units=dense_units,
#                          activation=activation, bidirectional=bidirectional, num_lstm_layers=num_lstm_layers)
# model.summary()

In [ ]:
# # Compile the model
# optimizer = tf.keras.optimizers.Adam()
# model.compile(optimizer=optimizer, loss=custom_mae_loss, metrics=[dir_acc])     # TODO: compare with MSE

In [ ]:
def compile_model(
  input_shape,
  lstm_units=50,
  dropout_rate=0.2,
  dense_units=25,
  activation='relu',
  output_units=1,
  bidirectional=False,
  num_lstm_layers=2,
  optimizer=tf.keras.optimizers.Adam(),
  loss=percent_error_squared_adjusted,
  metrics=[dir_acc, percent_error_true, error_direction, mse_loss, mae_loss],
):
  if os.path.exists("lstm_train_model.keras"):
    os.remove("lstm_train_model.keras")
  if os.path.exists("lstm_val_model.keras"):
    os.remove("lstm_val_model.keras")

  model = build_lstm_model(input_shape, lstm_units=lstm_units,
                         dropout_rate=dropout_rate, dense_units=dense_units,
                         activation=activation, bidirectional=bidirectional, num_lstm_layers=num_lstm_layers)
  model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
  #model.summary()
  return model

def train_model(
  model,
  train_sequences,
  train_labels,
  validation_sequences,
  validation_labels,
  epochs: int,
  batch_size: int,
):

  checkpoint_callback_val = ModelCheckpoint(
      "lstm_val_model.keras",
      monitor="val_loss", #"val_loss",
      save_best_only=True,
      mode="min",  # Minimize the monitored metric
      verbose=1,
  )

  model.fit(
    train_sequences,
    train_labels,
    validation_data=(validation_sequences, validation_labels),
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True, #TODO this takes care of shuffling
    callbacks=[checkpoint_callback_val],
  )
  print("Training complete")

def evaluate_model(
  model,
  test_sequences,
  test_labels,
):

  if os.path.exists("lstm_val_model.keras"):
    model.load_weights("lstm_val_model.keras")
  else:
    print("No model found")
    return

  accuracy = model.evaluate(test_sequences, test_labels)[1]
  predictions = model.predict(test_sequences)
  r2 = r2_score(test_labels[:, 1], predictions[:, 0])

  return predictions, accuracy, r2


In [21]:
optimizer=tf.keras.optimizers.Adam()
input_shape = train_data.shape[1:]
lstm_units = 50
dropout_rate = 0.2
dense_units = 25
activation = 'relu'
bidirectional = True
num_lstm_layers = 3
model = compile_model(input_shape, lstm_units, dropout_rate, dense_units, activation, bidirectional, num_lstm_layers, optimizer=optimizer, loss=percent_error_unnormalized)
train_model(model, train_data, train_labels, dev_data, dev_labels, epochs=5, batch_size=128)
predictions, accuracy, r2 = evaluate_model(model, test_data, test_labels)

Epoch 1/5
3754/3756 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - dir_acc: 0.5136 - error_direction: 0.0221 - loss: 0.0189 - mae_loss: 0.6231 - mse_loss: 0.7770 - percent_error_true: 25.8055
Epoch 1: val_loss improved from inf to 0.01220, saving model to lstm_val_model.keras
3756/3756 ━━━━━━━━━━━━━━━━━━━━ 104s 26ms/step - dir_acc: 0.5136 - error_direction: 0.0221 - loss: 0.0189 - mae_loss: 0.6231 - mse_loss: 0.7769 - percent_error_true: 25.8171 - val_dir_acc: 0.5051 - val_error_direction: 0.0401 - val_loss: 0.0122 - val_mae_loss: 0.5621 - val_mse_loss: 0.7531 - val_percent_error_true: -87.2140
Epoch 2/5
3755/3756 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - dir_acc: 0.5334 - error_direction: 0.0226 - loss: 0.0166 - mae_loss: 0.5685 - mse_loss: 0.6754 - percent_error_true: 37.6232
Epoch 2: val_loss improved from 0.01220 to 0.01216, saving model to lstm_val_model.keras
3756/3756 ━━━━━━━━━━━━━━━━━━━━ 99s 26ms/step - dir_acc: 0.5334 - error_direction: 0.0226 - loss: 0.0166 - mae_loss: 0.5685 - mse_loss: 0.67

In [ ]:
#drive.mount('/content/drive')

epochs = 10
num_models=10

def hyperparam_search(
    num_models: int,
    train_sequences,
    train_labels,
    val_sequences,
    val_labels,
    test_sequences,
    test_labels,
    optimizer=tf.keras.optimizers.Adam(),
    loss=percent_error_squared_unnormalized,
):

  input_shape = train_sequences.shape[1:]
  lstm_units_arr = [30, 50, 70, 90] # 50
  dense_units_arr = [15, 25, 35, 45, 55] # 25
  num_layers_arr = [3, 4, 5, 6] # 3
  dropout_arr = [0, 0.05, 0.10, 0.20, 0.3, 0.4, 0.5, 0.6] # 0.20
  batch_size_arr = [32, 64, 128]

  best_accuracy = 0
  best_params = None

  #with open('/content/drive/MyDrive/hyperparams.txt', 'w') as f:
  for i in range(num_models):
    try:
      batch_size = random.choice(batch_size_arr)
      lstm_units = random.choice(lstm_units_arr)
      dense_units = random.choice(dense_units_arr)
      num_layers = random.choice(num_layers_arr)
      dropout = random.choice(dropout_arr)

      model = compile_model(input_shape, lstm_units, dropout, dense_units, activation, bidirectional, num_layers, optimizer=optimizer, loss=loss)
      train_model(model, train_sequences, train_labels, val_sequences, val_labels, epochs, batch_size)
      predictions, accuracy, r2 = evaluate_model(model, test_sequences, test_labels)

      if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = (input_shape, lstm_units, dense_units, ff_dim, num_layers, dropout)

      write = f"R-squared: {r2}, accuracy: {accuracy}, batch_size: {batch_size}, lstm_units: {lstm_units}, dense_units: {dense_units}, num_layers: {num_layers}, dropout: {dropout}"
      print(write)
      # f.write(write + "\n")
      # f.flush()

    except Exception as e:
      print(f"Exception: {e}, continuing")

    best = f"best params: {best_params}, best accuracy: {best_accuracy}"
    print(best)
    # f.write(best + "\n")
    # f.flush()


In [ ]:
hyperparam_search(
    num_models,
    train_data,
    train_labels,
    dev_data,
    dev_labels,
    test_data,
    test_labels,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
)

In [ ]:
# # Define a callback to save the best model
# checkpoint_callback_train = ModelCheckpoint(
#     "lstm_train_model.keras",  # Filepath to save the best model
#     monitor="dir_acc",  #"loss",  # Metric to monitor
#     save_best_only=True,  # Save only the best model
#     mode="max",  # Minimize the monitored metric
#     verbose=1,  # Display progress
# )

# # Define a callback to save the best model
# checkpoint_callback_val = ModelCheckpoint(
#     "lstm_val_model.keras",  # Filepath to save the best model
#     monitor="val_dir_acc", #"val_loss",  # Metric to monitor
#     save_best_only=True,  # Save only the best model
#     mode="max",  # Minimize the monitored metric
#     verbose=1,  # Display progress
# )

In [ ]:
# BATCH_SIZE = 64
# EPOCHS = 100
# model.fit(train_sequences, train_labels,
#           validation_data=(validation_sequences, validation_labels),
#           epochs=EPOCHS,
#           batch_size=BATCH_SIZE,
#           shuffle=True,
#           callbacks=[checkpoint_callback_train, checkpoint_callback_val])  # removed get_lr_callback(batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
# # Load Weights
# model.load_weights("lstm_val_model.keras")

# # Make predictions
# accuracy = model.evaluate(test_sequences, test_labels)[1]
# print(accuracy)

# # Calculate additional metrics as needed
# from sklearn.metrics import r2_score

# predictions = model.predict(test_sequences)
# r2 = r2_score(test_labels[:, 1], predictions[:, 0])
# print(f"R-squared: {r2}")

In [ ]:
# end_time = time.time()

# elapsed_time = end_time - start_time

# # Convert elapsed time to hours, minutes, and seconds
# hours, rem = divmod(elapsed_time, 3600)
# minutes, seconds = divmod(rem, 60)

# print(f"Total runtime: {int(hours):02}:{int(minutes):02}:{int(seconds):02}")

Total runtime: 00:04:58
